In [1]:
!pip install pyomo

     |████████████████████████████████| 9.4MB 2.6MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 256kB 42.8MB/s 
     |████████████████████████████████| 163kB 44.2MB/s 


**Importing various libraries required for the assignment**

In [2]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

**Declaring various variables**
1 . **days** : List containing days Monday to Saturday,
2 . **slots** : List containing three batches,
3 . **lectures** : List containing five lectures for a particular batch,
4 . **Teachers** : List containing five teacher names,
5 . **slot_in_Batch** : Dictionary containing slots as keys and lectures as values,
6 . **day_slots** : Dictionary containing day as keys and slots as values,
7 . **pref** : Dictionary as described in the problem statement containing preferences of each teachers on each day of a week .

In [3]:
days = ['Mon','Tue','Wed','Thu','Fri','Sat']
slots = ['Batch1','Batch2','Batch3']
lectures = ['lec'+str(i) for i in range(1,6)]
slot_in_Batch={slot: lectures for slot in slots}
day_slots = {day: slots for day in days}
Teachers = ['T'+ str(i) for i in range(1,6)]
pref = {'T1':{'Mon':10,'Tue':7,'Wed':9,'Thu':5,'Fri':4,'Sat':1},'T2':{'Mon':6,'Tue':10,'Wed':8,'Thu':7,'Fri':3,'Sat':2},'T3':{'Mon':5,'Tue':6,'Wed':7,'Thu':10,'Fri':9,'Sat':8},'T4':{'Mon':3,'Tue':8,'Wed':10,'Thu':7,'Fri':8,'Sat':3},'T5':{'Mon':4,'Tue':6,'Wed':9,'Thu':8,'Fri':10,'Sat':1}}

**Declaring ConcreteModel and adding a binary variable 'teachers' to the model, it defines if a teacher is available at a particular lecture at a particular slot at a particular day**

In [4]:
model = ConcreteModel()
model.teachers = Var(((teacher,day,slot,lect) for teacher in Teachers for day in days for slot in day_slots[day] for lect in slot_in_Batch[slot]),within=Binary, initialize=0)

**Declaring the objective of the optimization problem and adding it to the model**

In [5]:
def obj_rule(m):
  h = len(Teachers)
  return sum(pref[teacher][day]*m.teachers[teacher,day,slot,batch] for teacher in Teachers for day in days for slot in day_slots[day] for batch in slot_in_Batch[slot])
model.obj = Objective(rule=obj_rule, sense=maximize)

**Declaring empty list of constraints for the model**

In [6]:
model.d= ConstraintList()

**Adding various constraints to the list of constraints**

In [7]:
#Constraint-"Every batch of students must get at least 5 lectures of each subject, every week".
for teacher in Teachers:
  for batch in slots:
      model.d.add(sum(model.teachers[teacher,day,batch,lect] for day in days for lect in slot_in_Batch[batch])>=5)
#Constraint-"There will not be same teacher in same slot for three batches of students".
for teacher in Teachers:
  for day in days:
    for lect in lectures:
      model.d.add(sum(model.teachers[teacher,day,slot,lect] for slot in day_slots[day])<=1)
#Constraint-"There are 5 lectures each day for all batches".
for day in days:
      for slot in day_slots[day]:
        for batch in slot_in_Batch[slot]:
          model.d.add(sum(model.teachers[teacher,day,slot,batch] for teacher in Teachers )==1)
#Constraint-"Teachers are not allowed to have more than 2 lectures in a day for a class".
for teacher in Teachers:
    for day in days:
       for slot in day_slots[day]:
          model.d.add(sum(model.teachers[teacher,day,slot,batch] for batch in slot_in_Batch[slot] )<=2)

**Declaring the solver and obtainining the result of the optimization problem using that solver**

In [8]:
opt = SolverFactory('cbc')
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt=opt) 

**Getting the final timetable after the completion of the optimization**

In [9]:
def get_time_table(teachers):
    week_table = {day: {slot: {batch:[] for batch in slot_in_Batch[slot]} for slot in day_slots[day]} for day in days}
    for teacher in Teachers:
        for day in days:
            for slot in day_slots[day]:
              for batch in slot_in_Batch[slot]:
                  if teachers[teacher, day, slot,batch].value == 1:
                        week_table[day][slot][batch].append(teacher)
    return week_table
week_table = get_time_table(model.teachers)
print(week_table)

{'Mon': {'Batch1': {'lec1': ['T2'], 'lec2': ['T3'], 'lec3': ['T1'], 'lec4': ['T2'], 'lec5': ['T1']}, 'Batch2': {'lec1': ['T1'], 'lec2': ['T2'], 'lec3': ['T3'], 'lec4': ['T3'], 'lec5': ['T2']}, 'Batch3': {'lec1': ['T3'], 'lec2': ['T1'], 'lec3': ['T2'], 'lec4': ['T1'], 'lec5': ['T3']}}, 'Tue': {'Batch1': {'lec1': ['T4'], 'lec2': ['T2'], 'lec3': ['T2'], 'lec4': ['T1'], 'lec5': ['T4']}, 'Batch2': {'lec1': ['T1'], 'lec2': ['T1'], 'lec3': ['T4'], 'lec4': ['T2'], 'lec5': ['T2']}, 'Batch3': {'lec1': ['T2'], 'lec2': ['T4'], 'lec3': ['T1'], 'lec4': ['T4'], 'lec5': ['T1']}}, 'Wed': {'Batch1': {'lec1': ['T4'], 'lec2': ['T4'], 'lec3': ['T1'], 'lec4': ['T5'], 'lec5': ['T1']}, 'Batch2': {'lec1': ['T1'], 'lec2': ['T5'], 'lec3': ['T5'], 'lec4': ['T1'], 'lec5': ['T4']}, 'Batch3': {'lec1': ['T5'], 'lec2': ['T1'], 'lec3': ['T4'], 'lec4': ['T4'], 'lec5': ['T5']}}, 'Thu': {'Batch1': {'lec1': ['T3'], 'lec2': ['T5'], 'lec3': ['T5'], 'lec4': ['T2'], 'lec5': ['T3']}, 'Batch2': {'lec1': ['T4'], 'lec2': ['T3'], '